# 연습 - 종목정보(코드,가격), 계좌잔고확인, 주문, 스케쥴링

## 주의사항
-------------------------------------
* 파이썬은 3.6 32비트 버전을 설치
* 프로젝트 경로 및 폴더명은 공백과 한글이 없도록 설치
* 사이보스는 관리자 권한으로 수행
* 주피터 노트북이 너무 느리면, VS Code에서 파이썬 플러그인을 설치해서 개발
* VS Code Python plugin에서 주피터 서버를 띄워줌. 수행속도는 차이가 없으나 코딩화면에서 반응 속도 빠름.
-------------------------------------

## 종목정보

- [x] @mentions, #refs, [links](), **formatting**, and <del>tags</del> supported
- [x] list syntax required (any unordered or ordered list supported)
- [x] this is a complete item
- [ ] this is an incomplete item



-------------------------------------

	종목번호	F-KOSDAQ150	(연결선물코드 10100)
			
	현재 시각	2020/07/28 14:22:00
	갱신 시각	2020/07/28 14:01:38
			
	20일 종가 내역	Date	Close
		2020/06/30 15:30:00	1256.61
		...
		2020/07/27 15:30:00	1357.23
			
	20일 이동 평균 (종가기준)		1305.58
			
	시초가		1365.03
	최고가		1379.11
	최저가		1362.54
	현재가		1369.69
	IBS     (일일변동폭 대비 종가 수준)		43.15
			
	월모멘텀 : 20이평보다 현재가가 비싼가	T
	단기하락 : IBS가 현재 25%이하인가	    F
	-----------------------------------------
	매수여부		보류


In [3]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
import datetime
 
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
g_objFutureMgr = win32com.client.Dispatch("CpUtil.CpKFutureCode")
g_objFutureMaster = win32com.client.Dispatch("Dscbo1.FutureMst")

g_objCpTrade0721 = win32com.client.Dispatch('CpTrade.CpTd0721F')
g_objCpTrade0732 = win32com.client.Dispatch('CpTrade.CpTd0732')
g_objCpTrade6722 = win32com.client.Dispatch('CpTrade.CpTd6722') # 선물/옵션신규주문가능수량조회
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # 주문 관련 초기화
    ret = g_objCpTrade.TradeInit(0)
    if (ret != 0):
        print("주문 초기화 실패, 오류번호 ", ret)
        return False
 
    return True
 
# CpFutureWeek: 선물 일자별
class CpFutureWeek:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("Dscbo1.FutureWeek1")
 
    def request(self, code, retList):
        self.objRq.SetInputValue(0, code)
 
        datacnt= 0
        while True:
            self.objRq.BlockRequest()
 
            rqStatus = self.objRq.GetDibStatus()
            rqRet = self.objRq.GetDibMsg1()
            if rqStatus != 0:
                print("통신상태", rqStatus, rqRet)
                return False
 
            cnt = self.objRq.GetHeaderValue(0)
 
            for i in range(cnt):
                item = {}
                item['일자'] = self.objRq.GetDataValue(0, i)
                item['시가'] = self.objRq.GetDataValue(1, i)
                item['고가'] = self.objRq.GetDataValue(2, i)
                item['저가'] = self.objRq.GetDataValue(3, i)
                item['종가'] = self.objRq.GetDataValue(4, i)
                item['전일대비'] = self.objRq.GetDataValue(5, i)
                item['누적거래량'] = self.objRq.GetDataValue(6, i)
                item['거래대금'] = self.objRq.GetDataValue(7, i)
                item['미결제약정'] = self.objRq.GetDataValue(8, i)
 
                retList.append(item)
            # end of for
 
            datacnt += cnt
            if self.objRq.Continue == False :
                break
        # end of while
 
 
        # for item in  retList:
        #     data = ''
        #     for key, value in item.items():
        #         if (type(value) == float):
        #             data += '%s:%.2f' % (key, value)
        #         elif (type(value) == str):
        #             data += '%s:%s' % (key, value)
        #         elif (type(value) == int):
        #             data += '%s:%d' % (key, value)
 
        #         data += ' '
        #     print(data)
        # return True
 
 
if __name__ == "__main__":
    if False == InitPlusCheck() :
        exit()
    print("================================")

    # 20일 이동평균 + IBS 구하기
    code = "10600" #코스닥 연결선물코드
    obj = CpFutureWeek()
    retList = []
    obj.request(code, retList) 

    startAmt  = 0.0
    endAmt    = 0.0
    topAmt    = 0.0
    bottomAmt = 0.0    

    ibs       = 0.0

    count = 0
    ma20sum = 0.0
    for item in  retList:
        data = ''

        for key, value in item.items():
            if key == '종가' or key == '일자':
                if (type(value) == float):
                    data += '%s:%.2f' % (key, value)
                    ma20sum = ma20sum + value
                elif (type(value) == str):
                    data += '%s:%s' % (key, value)
                elif (type(value) == int):
                    data += '%s:%d' % (key, value)
        
            data += ' '


        print(data)
        count = count + 1
        if count >= 20 : break   

        if count == 1: #첫레코드는 당일, 당일의 시가 고가 저가 현재가를 가져옴
            nowDate    = item['일자']
            startAmt   = item['시가'] 
            topAmt     = item['고가']
            bottomAmt  = item['저가'] 
            endAmt     = item['종가'] 
            beforeRate = item['전일대비']
            transCnt   = item['누적거래량'] 
            transAmt   = item['거래대금'] 
            noCnmt     = item['미결제약정']

    ma20 = ma20sum / 20
    ibs  = (endAmt - bottomAmt) / (topAmt - bottomAmt) * 100

    # # 종목증거금율
    # value = g_objCodeMgr.GetStockMarginRate( "000660" )
    # print("000660 증거금율 : ", value)
    # print("==========================================")  



    print("==========================================")  
    # 최근 선물코드, 두번째 선물코드를 가져온다.
    fstKFCode = ''
    sndKFCode = ''
    for i in range(g_objFutureMgr.GetCount()):
        code = g_objFutureMgr.GetData(0, i)
        name = g_objFutureMgr.GetData(1, i)
        if (code[0] == '4') :   # spread skip
            continue
        if (code[0] == '10100') : #연결선물 skip
            continue
        if i == 0 : fstKFCode = code
        if i == 1 : sndKFCode = code        

    # 최근 선물코드로 잔존일을 가져온다.
    g_objFutureMaster.SetInputValue(0, fstKFCode)
    # g_objFutureMaster.SetInputValue(0, '106Q9')
    g_objFutureMaster.BlockRequest()
    rqStatus = g_objFutureMaster.GetDibStatus()
    rqRet = g_objFutureMaster.GetDibMsg1()
    # print("통신상태", rqStatus, rqRet)        
    remainDate = g_objFutureMaster.GetHeaderValue(8) - 1 # 잔존일

    print("당기선물코드         : %s" % fstKFCode)
    print("차기선물코드         : %s" % sndKFCode)
    print("당기선물잔존일       : %d" % remainDate)    

    print("==========================================")   
    # 현재일시
    nowTimeStr = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("현재 일시            : %s" % nowTimeStr)

    print("==========================================")   
    print("20일 이동평균(종가)               :%.2f" % ma20)
    print("==========================================")   
    print("당일 최고가                       :%.2f" % topAmt)
    print("당일 최저가                       :%.2f" % bottomAmt)
    print("당일 현재가                       :%.2f" % endAmt)
    print("==========================================")
    print("IBS(일일변동폭 대비 현재가 수준)  :%7.2f" % ibs)
    print("==========================================")
    f1 = 0.0
    f2 = 0.0
    buyForce = 0
    if ma20 < endAmt : 
        f1 = 1.0
    else:
        f1 = 0.0

    if ibs < 25 : 
        f2 = 1.0 
    else:
        f2 = 0.0
    buyForce = ((f1 + f2) ** 2.321) * 20
    print("월모멘텀(20일 이평 < 현재가)      :   %.2f" % f1)
    print("단기하락(IBS < 25 퍼센트)         :   %.2f" % f2)
    print("매수강도(월모멘텀+단기하락)^2.321 :     %.f" % buyForce)
    print("==========================================")
    


    print("=================================================================================")    
    accNum = g_objCpTrade.AccountNumber[0]  # 계좌번호
    print("계좌번호 : %s" % accNum)    

    g_objCpTrade0721.SetInputValue(0,accNum)
    g_objCpTrade0721.SetInputValue(1,"50")
    # g_objCpTrade0721.SetInputValue(2,"106Q9")
    
    g_objCpTrade0721.SetInputValue(3,20)
    # 0 - (string)계좌번호
    # 1 - (string)상품관리구분[defaut:50]
    # 2 - (string) 종목코드[default:""]
    # 3 - (long) 요청건수[default:10] - 최대20개
    
    g_objCpTrade0721.BlockRequest()
 

    rqStatus = g_objCpTrade0721.GetDibStatus()
    rqRet = g_objCpTrade0721.GetDibMsg1()
    print("g_objCpTrade0721 통신상태", rqStatus, rqRet)

    deposits = g_objCpTrade0721.GetHeaderValue(3 ) # 예탁액, 주문가능금액
        

    print("3 - (longlong)현금결제예탁액        : %d" % g_objCpTrade0721.GetHeaderValue(3 ))
    print("4 - (longlong)대용결제예탁액        : %d" % g_objCpTrade0721.GetHeaderValue(4 ))
    print("5 - (longlong)총결제예탁액          : %d" % g_objCpTrade0721.GetHeaderValue(5 ))
    print("6 - (longlong)현금위탁증거금        : %d" % g_objCpTrade0721.GetHeaderValue(6 ))
    print("7 - (longlong)대용위탁증거금        : %d" % g_objCpTrade0721.GetHeaderValue(7 ))
    print("8 - (longlong)총위탁증거금          : %d" % g_objCpTrade0721.GetHeaderValue(8 ))
    print("9 - (longlong)현금주문가능액        : %d" % g_objCpTrade0721.GetHeaderValue(9 ))
    print("10 - (longlong)대용주문가능액       : %d" % g_objCpTrade0721.GetHeaderValue(10))
    print("11 - (longlong)총주문가능액         : %d" % g_objCpTrade0721.GetHeaderValue(11))
    print("12 - (longlong)현금인출가능액       : %d" % g_objCpTrade0721.GetHeaderValue(12))
    print("13 - (longlong)대용인출가능액       : %d" % g_objCpTrade0721.GetHeaderValue(13))
    print("14 - (longlong)총인출가능액         : %d" % g_objCpTrade0721.GetHeaderValue(14))
    print("15 - (longlong)순위험증거금         : %d" % g_objCpTrade0721.GetHeaderValue(15))
    print("17 - (longlong)체결분순익           : %d" % g_objCpTrade0721.GetHeaderValue(17))
    print("18 - (longlong)미체결주문분         : %d" % g_objCpTrade0721.GetHeaderValue(18))
    print("20 - (longlong)현금미결제예탁액     : %d" % g_objCpTrade0721.GetHeaderValue(20))
    print("21 - (longlong)대용미결제예탁액     : %d" % g_objCpTrade0721.GetHeaderValue(21))
    print("22 - (longlong)총미결제예탁액       : %d" % g_objCpTrade0721.GetHeaderValue(22))
    print("23 - (longlong)현금결재순손실       : %d" % g_objCpTrade0721.GetHeaderValue(23))
    print("24 - (longlong)총결재순손실         : %d" % g_objCpTrade0721.GetHeaderValue(24))
    print("25 - (long) 수신개수                : %d" % g_objCpTrade0721.GetHeaderValue(25))
    print("26- (long) 매도신규주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(26))
    print("27- (long) 매도청산주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(27))
    print("28- (long) 매수신규주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(28))
    print("29- (long) 매수청산주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(29))
    print("30- (longlong) 체결분순손실         : %d" % g_objCpTrade0721.GetHeaderValue(30))
    print("31 - (longlong) C등급총위험증거금   : %d" % g_objCpTrade0721.GetHeaderValue(31))
    print("32 - (longlong) C등급총위험가결산   : %d" % g_objCpTrade0721.GetHeaderValue(32))
    print("33 - (longlong) D등급총위험증거금   : %d" % g_objCpTrade0721.GetHeaderValue(33))
    print("34 - (longlong) D등급총위험가결산   : %d" % g_objCpTrade0721.GetHeaderValue(34))
    print("35 - (longlong) E등급총위험증거금   : %d" % g_objCpTrade0721.GetHeaderValue(35))
    print("36 - (longlong) E등급총위험가결산   : %d" % g_objCpTrade0721.GetHeaderValue(36))



    print("==========================================")
    print("계좌번호 : %s" % accNum)    
    g_objCpTrade6722.SetInputValue(0,accNum)
    g_objCpTrade6722.SetInputValue(1,"106Q9") # 종목
    g_objCpTrade6722.SetInputValue(2,0) # 주문가격
    g_objCpTrade6722.SetInputValue(3,"2") # 주문유형(1:지정가,2:시장가,3:조건부지정가,4:최유리지정가)
    g_objCpTrade6722.SetInputValue(4,"50") # 상품종목
    g_objCpTrade6722.SetInputValue(5,"Y") # 수수료포함여부

    
    g_objCpTrade6722.BlockRequest()


    rqStatus = g_objCpTrade6722.GetDibStatus()
    rqRet = g_objCpTrade6722.GetDibMsg1()
    print("g_objCpTrade6722 통신상태", rqStatus, rqRet)

    availableQty = g_objCpTrade6722.GetHeaderValue(29) # 매수가능수량
    sellableQty  = g_objCpTrade6722.GetHeaderValue(19) # 매도가능수량
    print("2 - (longlong)현금주문전주문가능금액   %d" % g_objCpTrade6722.GetHeaderValue(2 ))
    print("3 - (longlong)대용주문전주문가능금액   %d" % g_objCpTrade6722.GetHeaderValue(3 ))
    print("4 - (longlong)총액주문전주문가능금액   %d" % g_objCpTrade6722.GetHeaderValue(4 ))
    print("11 - (longlong)현금매도신규분증거금    %d" % g_objCpTrade6722.GetHeaderValue(11))
    print("12 - (longlong)대용매도신규분증거금    %d" % g_objCpTrade6722.GetHeaderValue(12))
    print("13 - (longlong)총액매도신규분증거금    %d" % g_objCpTrade6722.GetHeaderValue(13))
    print("14 - (longlong)현금매도주문후가능금액  %d" % g_objCpTrade6722.GetHeaderValue(14))
    print("15 - (longlong)대용매도주문후가능금액  %d" % g_objCpTrade6722.GetHeaderValue(15))
    print("16 - (longlong)총액매도주문후가능금액  %d" % g_objCpTrade6722.GetHeaderValue(16))
    print("17 - (long)매도보유포지션수량          %d" % g_objCpTrade6722.GetHeaderValue(17))
    print("18 - (long)매도청산주문가능수량        %d" % g_objCpTrade6722.GetHeaderValue(18))
    print("19 - (long)매도신규주문가능수량        %d" % g_objCpTrade6722.GetHeaderValue(19))
    print("20 - (long) 매도총주문가능수량         %d" % g_objCpTrade6722.GetHeaderValue(20))
    print("21 - (longlong)현금매수신규주문증거금  %d" % g_objCpTrade6722.GetHeaderValue(21))
    print("22 - (longlong)대용매수신규주문증거금  %d" % g_objCpTrade6722.GetHeaderValue(22))
    print("23 - (longlong)총액매수신규주문증거금  %d" % g_objCpTrade6722.GetHeaderValue(23))
    print("24 - (longlong)현금매수주문후가능금액  %d" % g_objCpTrade6722.GetHeaderValue(24))
    print("25 - (longlong)대용매수주문후가능금액  %d" % g_objCpTrade6722.GetHeaderValue(25))
    print("26 - (longlong)총액매수주문후가능금액  %d" % g_objCpTrade6722.GetHeaderValue(26))
    print("27 - (long)매수보유포지션수량          %d" % g_objCpTrade6722.GetHeaderValue(27))
    print("28 - (long)매수청산주문가능수량        %d" % g_objCpTrade6722.GetHeaderValue(28))
    print("29 - (long)매수신규주문가능수량        %d" % g_objCpTrade6722.GetHeaderValue(29))
    print("30 - (long) 매수총주문가능수량         %d" % g_objCpTrade6722.GetHeaderValue(30))
        

    print("예수금                            :   %d" % deposits)    
    print("목표주문액 (예수금 * 매수강도)    :   %d" % round(deposits * buyForce ,0))
    print("가능포지션 (현재가 * 가능수량)    :   %d" % round(endAmt * availableQty * 10000,0))

    print("==========================================")
    print("매수 계약수                       :      3")
    print("최대 계약수                       :     50")
    print("max(매수,최대)                    :      3")
    print("주문가능 수량                     :      5")
    print("당기선물 잔존일이 0이면 매수하지 않는다.  ")
    print("")   
    print("")       

정상: 관리자권한으로 실행된 프로세스입니다.
일자:20200731    종가:1267.90     
일자:20200730    종가:1263.40     
일자:20200729    종가:1264.70     
일자:20200728    종가:1269.50     
일자:20200727    종가:1253.00     
일자:20200724    종가:1240.20     
일자:20200723    종가:1253.10     
일자:20200722    종가:1236.00     
일자:20200721    종가:1230.30     
일자:20200720    종가:1213.10     
일자:20200717    종가:1216.10     
일자:20200716    종가:1201.40     
일자:20200715    종가:1213.40     
일자:20200714    종가:1207.40     
일자:20200713    종가:1215.30     
일자:20200710    종가:1208.30     
일자:20200709    종가:1205.80     
일자:20200708    종가:1198.70     
일자:20200707    종가:1192.10     
일자:20200706    종가:1187.40     
당기선물코드         : 106Q9
차기선물코드         : 106QC
당기선물잔존일       : 41
현재 일시            : 2020-07-31 17:41:48
20일 이동평균(종가)               :1226.86
당일 최고가                       :1272.60
당일 최저가                       :1262.60
당일 현재가                       :1267.90
IBS(일일변동폭 대비 현재가 수준)  :  53.00
월모멘텀(20일 이평 < 현재가)      :   1.00
단기하락(IBS < 25 퍼센트)         :   0.00
매